In [170]:
import numpy as np
from numpy import linalg as LA

def example_from_article_to_generate_coordinate_matrix():
    X = np.matrix('0.5 0.5; 0.5 -0.5; -0.5 -0.5; -0.5 0.5')
    X_T = X.getT()
    G = X*X_T
    W, V = LA.eig(G)
    index_nul = []
    k = 0
    for i in range(0, len(W)):
        if W[i] < pow(10, -15):
            index_nul.append(i)
            V = np.delete(V, i-k, axis=1)
            k += 1

    if index_nul:
        W = np.delete(W, index_nul, axis=0)
        
#     print(W)
#     print(V)
    
    L = np.diag((np.sqrt(W)))
    print(L)
    print(V*L)
    
example_from_article_to_generate_coordinate_matrix()

[[1. 0.]
 [0. 1.]]
[[ 0.70710678  0.        ]
 [ 0.          0.70710678]
 [-0.70710678  0.        ]
 [ 0.         -0.70710678]]


### Parse .mol file

In [171]:
import re


van_der_Waals_radii = [0.001, 31, 28, 128, 96, 84, 73, 71, 66, 57, 58, 166, 141, 121, 111, 107, 105, 102, 106, 203, 176, 170, 160,
            153, 139, 139, 132, 126, 124, 132, 122, 122, 120, 119, 120, 120, 116, 220, 195, 190, 175, 164, 154, 147,
            146, 142, 139, 145, 144, 142, 139, 139, 138, 139, 140, 244, 215, 207, 204, 203, 201, 199, 198, 196, 194,
            192, 192, 189, 190, 187, 187, 175, 170, 162, 151, 144, 141, 136, 136, 132, 145, 146, 148, 140, 150, 150,
            260, 221, 215, 206, 200, 196, 190, 187, 180, 169]

elements = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", 
            "P", "S", "Cl", "Ar", "K", "Ca"]


def read_file(filename):
    lines = []
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    is_counts_line = False
    is_atom_block = 0
    is_bond_block = 0
    atoms_number = 0
    bonds_number = 0
    atoms = []
    coordinates = []
    list_of_bonds = []

    for i in range(0, len(lines)):
        t = lines[i].lstrip().replace("\n", "")
        if not is_counts_line:
            counts_line = re.findall("V\d", t)
            if len(counts_line) == 1:
                t = t.split(" ")
                t = [x for x in t if x != ""]
                atoms_number = int(t[0])
                bonds_number = int(t[1])
                is_counts_line = True

        elif is_atom_block < atoms_number:
            atom_block_numbers = re.findall("\d", t)
            atom_block_element = [x for x in elements if len(re.findall(x, t)) == 1]
            if len(atom_block_element) == 1 and len(atom_block_numbers) > 2:
                t = t.split(" ")
                t = [x for x in t if x != ""]
                coordinates.append(float(t[0]))
                coordinates.append(float(t[1]))
                coordinates.append(float(t[2]))
                atoms.append(elements.index(t[3])+1)
                is_atom_block += 1

        elif is_bond_block < bonds_number:
            bond_block_letters = re.findall("[A-Za-z]", t)
            bond_block_numbers = re.findall("\d", t)
            if len(bond_block_letters) == 0 and len(bond_block_numbers) > 2:
                t = t.split(" ")
                t = [x for x in t if x != ""]
                distance = ((coordinates[3*(int(t[0])-1)] - coordinates[3*(int(t[1])-1)]) ** 2 + \
                             (coordinates[3*(int(t[0])-1) + 1] - coordinates[3*(int(t[1])-1) + 1]) ** 2 + \
                             (coordinates[3*(int(t[0])-1) + 2] - coordinates[3*(int(t[1])-1) + 2]) ** 2) ** 0.5
                list_of_bonds.append({"atom_1": int(t[0]), "atom_2": int(t[1]), \
                                      "number_of_bonds": int(t[2]), "distance": distance})
                is_bond_block += 1
    
    return atoms_number, bonds_number, atoms, coordinates, list_of_bonds


atoms_number, bonds_number, atoms, coordinates, list_of_bonds = read_file("output/metan.mol")

print(atoms_number, " ", bonds_number)
print(atoms)
print(coordinates)
print(list_of_bonds)

5   4
[6, 1, 1, 1, 1]
[0.9499, 0.0585, -0.0637, 0.5858, -0.9697, -0.0088, 0.5858, 0.6202, 0.7993, 0.5858, 0.5251, -0.9817, 2.0421, 0.0585, -0.0637]
[{'atom_1': 1, 'atom_2': 4, 'number_of_bonds': 1, 'distance': 1.0922492252228884}, {'atom_1': 1, 'atom_2': 5, 'number_of_bonds': 1, 'distance': 1.0922}, {'atom_1': 2, 'atom_2': 1, 'number_of_bonds': 1, 'distance': 1.0921437908993485}, {'atom_1': 3, 'atom_2': 1, 'number_of_bonds': 1, 'distance': 1.0921742992764478}]


### First step of algorithm - generation of distance bounds matrix

In [195]:
def adjust_default_upper_bound(atoms, list_of_bonds):
    list_of_bonds = sorted(list_of_bonds, key=lambda k: k["distance"], reverse=True) 
    atoms = list(set(atoms))
    radii = [van_der_Waals_radii[x-1] for x in atoms]
    radii = sorted(radii, reverse=True)
    bond = radii[0] + radii[1] if len(radii) > 1 else radii[0]
    bond += list_of_bonds[0]["distance"]
    bond *= 6
    return bond


def generate_distance_bounds_matrix(atoms, list_of_bonds, upper_bound=600):
    lower_bounds_matrix = np.asarray(np.zeros((len(atoms), len(atoms))))
    upper_bounds_matrix = np.asarray(np.zeros((len(atoms), len(atoms))))
    for i in range(0,len(lower_bounds_matrix[1])):
        for j in range(i+1, len(lower_bounds_matrix[1])):
            bond = False
            for a in list_of_bonds:
                if not bond and ((a["atom_1"]-1 == i and a["atom_2"]-1 == j) or \
                    (a["atom_2"]-1 == i and a["atom_1"]-1 == j)):
#                     lower_bounds_matrix[i,j] = ((coordinates[3*i] - coordinates[3*j]) ** 2 + \
#                                          (coordinates[3*i + 1] - coordinates[3*j + 1]) ** 2 + \
#                                          (coordinates[3*i + 2] - coordinates[3*j + 2]) ** 2) ** 0.5
                    
                    lower_bounds_matrix[i,j] = a["distance"]
                    upper_bounds_matrix[i,j] = lower_bounds_matrix[i,j]
                    bond = True
            if not bond:
                upper_bounds_matrix[i,j] = upper_bound
                lower_bounds_matrix[i,j] = van_der_Waals_radii[atoms[i]-1] + van_der_Waals_radii[atoms[j]-1]

    lower_bounds_matrix = lower_bounds_matrix + lower_bounds_matrix.T - np.diag(lower_bounds_matrix.diagonal())
    upper_bounds_matrix = upper_bounds_matrix + upper_bounds_matrix.T - np.diag(upper_bounds_matrix.diagonal())
    
    return lower_bounds_matrix, upper_bounds_matrix
    

upper_bound = adjust_default_upper_bound(atoms, list_of_bonds)
lower_bounds_matrix, upper_bounds_matrix = generate_distance_bounds_matrix(atoms, \
                                                                           list_of_bonds, upper_bound)
print(lower_bounds_matrix)
print(upper_bounds_matrix)

[[0.         1.09214379 1.0921743  1.09224923 1.0922    ]
 [1.09214379 0.         0.002      0.002      0.002     ]
 [1.0921743  0.002      0.         0.002      0.002     ]
 [1.09224923 0.002      0.002      0.         0.002     ]
 [1.0922     0.002      0.002      0.002      0.        ]]
[[  0.           1.09214379   1.0921743    1.09224923   1.0922    ]
 [  1.09214379   0.         510.55949535 510.55949535 510.55949535]
 [  1.0921743  510.55949535   0.         510.55949535 510.55949535]
 [  1.09224923 510.55949535 510.55949535   0.         510.55949535]
 [  1.0922     510.55949535 510.55949535 510.55949535   0.        ]]
